In [1]:
import pandas as pd

# Lire le fichier CSV principal
df = pd.read_csv('../../full_dataset.csv')
df = df.drop("date", axis=1)

# Identifier les coordonnées uniques (latitude, longitude)
coordinates = df[['latitude', 'longitude']].drop_duplicates()

# Liste pour stocker les dataframes échantillonnés
sampled_dfs = []

# Filtrer les données pour chaque bâtiment en fonction des coordonnées
for index, row in coordinates.iterrows():
    lat, long = row['latitude'], row['longitude']
    building_data = df[(df['latitude'] == lat) & (df['longitude'] == long)]
    
    # Calculer le nombre de lignes correspondant à 20% du dataframe
    num_rows = int(len(building_data) * 0.2)
    
    # Prendre les premières 20% des lignes
    sampled_df = building_data.head(num_rows)
    
    # Ajouter le dataframe échantillonné à la liste
    sampled_dfs.append(sampled_df)

# Fusionner tous les dataframes échantillonnés en un seul dataframe
data = pd.concat(sampled_dfs, ignore_index=True)

# Afficher un aperçu du dataframe final
print(data.head())


   production   latitude  longitude   vmp  imp   voc   isc  p_per_m2  p_max  \
0         0.8  48.575437   7.768668  27.3  7.7  33.3  8.17     143.0    210   
1        16.9  48.575437   7.768668  27.3  7.7  33.3  8.17     143.0    210   
2         1.4  48.575437   7.768668  27.3  7.7  33.3  8.17     143.0    210   
3         6.6  48.575437   7.768668  27.3  7.7  33.3  8.17     143.0    210   
4         0.3  48.575437   7.768668  27.3  7.7  33.3  8.17     143.0    210   

   panel_area  ...  wind_speed_10m_std  wind_speed_10m_min  wind_speed_10m_q1  \
0        1.72  ...            0.903114                 1.3              1.900   
1        1.72  ...            0.969186                 0.9              2.975   
2        1.72  ...            0.306945                 0.4              1.100   
3        1.72  ...            0.491844                 0.4              0.800   
4        1.72  ...            0.527737                 1.6              2.275   

   wind_speed_10m_q2  wind_speed_10m_q

In [2]:
data_train = data[data['building_id'] != 8]
data_test = data[data['building_id'] == 8]

data_train.shape, data_test.shape

((1919, 164), (250, 164))

In [3]:
target_column = 'production'

x_train = data_train.drop(target_column, axis=1)
y_train = data_train[target_column].values.reshape(-1, 1)

x_test = data_test.drop(target_column, axis=1)
y_test = data_test[target_column].values.reshape(-1, 1)

In [4]:
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error

In [5]:
x_scaler = MinMaxScaler(feature_range=(0, 1))
x_scaler.fit(x_train)

x_train_scaled = x_scaler.transform(x_train)
x_test_scaled = x_scaler.transform(x_test)

In [6]:
def get_windows(x, y, window_size):
    x_windows, y_windows = [], []

    for i in range(len(x) - window_size):
        x_window = x[i:i+window_size]
        y_window = y[i:i+window_size]

        x_window = np.hstack((x_window, y_window))

        x_windows.append(x_window)
        y_windows.append(y[i+window_size])

    return np.array(x_windows), np.array(y_windows)

In [7]:
x_train_windows, y_train_windows = get_windows(x_train_scaled, y_train, 10)
x_test_windows, y_test_windows = get_windows(x_test_scaled, y_test, 10)

In [8]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import time

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
else:
    print("No GPU found, using CPU instead.")

# Définition des niveaux des hyperparamètres
LSTM1_units = [192, 320]
LSTM1_activation = ['tanh', 'sigmoid']
DROPOUT1_rate = [0.2, 0.1]
LSTM2_units = [160, 208]
LSTM2_activation = ['tanh', 'sigmoid']
DROPOUT2_rate = [0.2, 0.1]
learning_rate = [0.001, 0.01]
epoch = 200

# Création du plan orthogonal L8(2^7)
orthogonal_array = [
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 1, 1, 1, 1, 1],
    [0, 1, 0, 0, 1, 1, 1],
    [0, 1, 1, 1, 0, 0, 0],
    [1, 0, 0, 1, 0, 1, 1],
    [1, 0, 1, 0, 1, 0, 0],
    [1, 1, 0, 1, 1, 0, 1],
    [1, 1, 1, 0, 0, 1, 0]
]

orthogonal_array = [
    [0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 1, 1, 1, 1],
    [0, 1, 1, 0, 0, 1, 1],
    [0, 1, 1, 1, 1, 0, 0],
    [1, 0, 1, 0, 1, 0, 1],
    [1, 0, 1, 1, 0, 1, 0],
    [1, 1, 0, 0, 1, 1, 0],
    [1, 1, 0, 1, 0, 0, 1],
    #[1, 1, 1, 1, 1, 1, 1],  # Ajout d'essais complémentaires
    #[1, 1, 1, 0, 0, 0, 0],
    #[1, 0, 0, 1, 1, 0, 0],
    #[1, 0, 0, 0, 0, 1, 1],
    #[0, 1, 0, 1, 0, 1, 0],
    #[0, 1, 0, 0, 1, 0, 1],
    #[0, 0, 1, 1, 0, 0, 1],
    #[0, 0, 1, 0, 1, 1, 0]
]

# Générer les combinaisons d'hyperparamètres à partir du plan orthogonal
param_combinations = []
for row in orthogonal_array:
    params = {
        'LSTM1_units': LSTM1_units[row[0]],
        'LSTM1_activation': LSTM1_activation[row[1]],
        'DROPOUT1_rate': DROPOUT1_rate[row[2]],
        'LSTM2_units': LSTM2_units[row[3]],
        'LSTM2_activation': LSTM2_activation[row[4]],
        'DROPOUT2_rate': DROPOUT2_rate[row[5]],
        'learning_rate': learning_rate[row[6]]
    }
    param_combinations.append(params)
# Prepare the data
# x_train_windows and y_train_windows should be pre-defined
# For the example, they are assumed to be loaded or created previously

# Function to create the model
def create_model(LSTM1_units, LSTM1_activation, DROPOUT1_rate, LSTM2_units, LSTM2_activation, DROPOUT2_rate, learning_rate):
    model = Sequential([
        LSTM(LSTM1_units, activation=LSTM1_activation, input_shape=x_train_windows.shape[1:], return_sequences=True),
        Dropout(DROPOUT1_rate),
        LSTM(LSTM2_units, activation=LSTM2_activation, return_sequences=False),
        Dropout(DROPOUT2_rate),
        Dense(64, activation='relu'),
        Dense(1, activation='linear')
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_absolute_error')
    return model

# Function to start training and tracking emissions
def start_training(model):
    start_time = time.time()
    history = model.fit(x=x_train_windows, y=y_train_windows, epochs=epoch, batch_size=128, validation_split=0.2, shuffle=False)
    training_duration = time.time() - start_time
    return history, training_duration

# DataFrame to store results
results_df = pd.DataFrame(columns=['LSTM1_units', 'LSTM1_activation', 'DROPOUT1_rate', 'LSTM2_units', 'LSTM2_activation', 'DROPOUT2_rate', 'learning_rate', 'loss', 'val_loss'])

# Iterate over the parameter combinations from the orthogonal array
best_params = None
best_loss = np.inf
total_combinations = len(param_combinations)
current_combination = 0

for params in param_combinations:
    current_combination += 1
    print(f"Training with parameters: {params} ({current_combination}/{total_combinations})")
    
    model = create_model(**params)
    
    try:
        history, training_duration = start_training(model)
        loss = min(history.history['loss'])
        val_loss = min(history.history['val_loss'])
        print(f"Loss: {loss}")
        print(f"Validation loss: {val_loss}")
        print(f"Training duration: {training_duration} seconds")
        
        # Create a temporary DataFrame with the current results
        temp_df = pd.DataFrame([{**params, 'loss': loss, 'val_loss': val_loss}])
        
        # Concatenate the temporary DataFrame with the existing DataFrame
        results_df = pd.concat([results_df, temp_df], ignore_index=True)
        
        # Plot and save the training history

        start = 10
        plt.figure()
        plt.plot(history.history['loss'][start:], label='train')
        plt.plot(history.history['val_loss'][start:], label='validation')
        plt.title(f"Loss Curves - Params: {params}")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
        plt.legend()
        plt.savefig(f"loss_curve_LSTM1_{params['LSTM1_units']}_LSTM1act_{params['LSTM1_activation']}_DROPOUT1_{params['DROPOUT1_rate']}_LSTM2_{params['LSTM2_units']}_LSTM2act_{params['LSTM2_activation']}_DROPOUT2_{params['DROPOUT2_rate']}_LR_{params['learning_rate']}_EPOCH_{epoch}.png")
        plt.close()
        
        # Check if this is the best model so far
        if val_loss < best_loss:
            best_loss = val_loss
            best_params = params
            print(f"New best model found: {best_params} with validation loss: {best_loss}")
    finally:
        pass

# Save the results DataFrame to a CSV file
results_df.to_csv(f"hyperparameter_results_LSTM1_{params['LSTM1_units']}_LSTM1act_{params['LSTM1_activation']}_DROPOUT1_{params['DROPOUT1_rate']}_LSTM2_{params['LSTM2_units']}_LSTM2act_{params['LSTM2_activation']}_DROPOUT2_{params['DROPOUT2_rate']}_LR_{params['learning_rate']}_EPOCH_{epoch}.csv", index=False)

print(f"Best parameters: {best_params}")
print(f"Best validation loss: {best_loss}")

2024-05-20 14:46:48.178358: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 14:46:49.276530: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-05-20 14:46:50.408758: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-05-20 14:46:50.443114: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. 

1 Physical GPUs, 1 Logical GPUs
Training with parameters: {'LSTM1_units': 192, 'LSTM1_activation': 'tanh', 'DROPOUT1_rate': 0.2, 'LSTM2_units': 160, 'LSTM2_activation': 'tanh', 'DROPOUT2_rate': 0.2, 'learning_rate': 0.001} (1/8)
Epoch 1/200


2024-05-20 14:46:53.623823: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:465] Loaded cuDNN version 8907


12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 35ms/step - loss: 31.7879 - val_loss: 16.5915
Epoch 2/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 25.6035 - val_loss: 13.2545
Epoch 3/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 22.8729 - val_loss: 12.9651
Epoch 4/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 22.1441 - val_loss: 11.8344
Epoch 5/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 20.9675 - val_loss: 11.1633
Epoch 6/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 19.6844 - val_loss: 9.5700
Epoch 7/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 18.8566 - val_loss: 9.1548
Epoch 8/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 18.0559 - val_loss: 8.9845
Epoch 9/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 17.5069 - val_loss: 8.8978
Epoch 10/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 17.0778 - val_loss: 8.6148
Epoch 11/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 16.6658 - val_loss: 8.5443
Epoch 12/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/s

/tmp/ipykernel_61741/1763495566.py:129: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, temp_df], ignore_index=True)


Epoch 1/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 3s 44ms/step - loss: 27.0796 - val_loss: 20.2753
Epoch 2/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 26.1616 - val_loss: 13.3386
Epoch 3/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 23.3284 - val_loss: 13.7924
Epoch 4/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 23.1723 - val_loss: 13.2728
Epoch 5/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 22.7051 - val_loss: 13.3028
Epoch 6/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 22.7196 - val_loss: 13.3053
Epoch 7/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 22.6639 - val_loss: 13.3309
Epoch 8/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 22.7261 - val_loss: 13.2777
Epoch 9/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 22.6623 - val_loss: 13.3065
Epoch 10/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 22.6500 - val_loss: 13.2347
Epoch 11/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 22.6413 - val_loss: 13.3503
Epoch 12/200
12/12 ━━━━━━━━━━━

I0000 00:00:1716209333.454376   61823 service.cc:145] XLA service 0x5e35c23bbb30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1716209333.454398   61823 service.cc:153]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2024-05-20 14:48:53.540715: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


 7/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 23.3406 

I0000 00:00:1716209338.803507   61823 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


12/12 ━━━━━━━━━━━━━━━━━━━━ 14s 539ms/step - loss: 32.3045 - val_loss: 17.5163
Epoch 2/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 26.3169 - val_loss: 13.6349
Epoch 3/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 23.3449 - val_loss: 13.0784
Epoch 4/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 22.4975 - val_loss: 13.3209
Epoch 5/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 22.5861 - val_loss: 13.2858
Epoch 6/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 22.5704 - val_loss: 13.1503
Epoch 7/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 22.4275 - val_loss: 12.9918
Epoch 8/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 22.2674 - val_loss: 12.6945
Epoch 9/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 21.8999 - val_loss: 11.9573
Epoch 10/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - loss: 21.1557 - val_loss: 10.8715
Epoch 11/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 20.1844 - val_loss: 10.2514
Epoch 12/200
12/12 ━━━━━━━━━━━━━━━━━━━━ 

In [9]:
print(results_df)

  LSTM1_units LSTM1_activation  DROPOUT1_rate LSTM2_units LSTM2_activation  \
0         192             tanh            0.2         160             tanh   
1         192             tanh            0.2         208          sigmoid   
2         192          sigmoid            0.1         160             tanh   
3         192          sigmoid            0.1         208          sigmoid   
4         320             tanh            0.1         160          sigmoid   
5         320             tanh            0.1         208             tanh   
6         320          sigmoid            0.2         160          sigmoid   
7         320          sigmoid            0.2         208             tanh   

   DROPOUT2_rate  learning_rate       loss   val_loss  
0            0.2          0.001   6.626012   4.535203  
1            0.1          0.010  23.260159  10.698532  
2            0.1          0.010  20.091169   9.377554  
3            0.2          0.001  10.332840   5.201952  
4            0.2 